In [1]:
import requests

In [2]:
class WikipediaAgent:
    
    SEARCH_URL = "https://en.wikipedia.org/w/api.php"
    PAGE_URL = "https://en.wikipedia.org/w/index.php"
    
    def __init__(self):
        pass
        
    def search(self, query: str):
        """
        Search Wikipedia for pages matching the query.
    
        Args:
            query: Search term
        
        Returns:
            Dictionary with search results
        """
        params = {
            "action": "query",
            "format": "json",
            "list": "search",
            "srsearch": query
        }
        headers = {
            "User-Agent": "WikipediaAgent/1.0 (https://github.com/slv-ai)"
        }
        
        response= requests.get(self.SEARCH_URL,params=params,headers=headers)
        response.raise_for_status()
        data = response.json()

        if not data["query"]["search"]:
            return None
        return [result["title"] for result in data["query"]["search"]]

    def get_page(self, title: str):
        """
        Get the raw content of a Wikipedia page.
    
        Args:
            title: Page title
        
        Returns:
            Raw page content as string
        """
        params = {
            "title": title,
            "action": "raw"
        }
        response = requests.get(self.PAGE_URL, params=params)
        if response.status_code == 404:
            return None
        return response.text


In [3]:
wiki=WikipediaAgent()

In [4]:
def search(query: str) -> list[str]:
    """Search Wikipedia for relevant article titles."""
    return wiki.search(query)

def get_page(title: str) -> str:
    """Retrieve the raw wiki content of a page."""
    return wiki.get_page(title)

In [5]:
from pydantic_ai import Agent

In [6]:
search_instructions = """
"You are a helpful Wikipedia research assistant. "
        "When answering questions, you should:\n"
        "1. First search Wikipedia for relevant articles\n"
        "2. Retrieve and read multiple relevant pages\n"
        "3. Synthesize information from the pages to answer the question\n"
        "4. Always include references to the Wikipedia pages you used\n"
        "5. Be concise but informative in your answers"
"""

In [7]:
wiki_agent = Agent(
    name = "wiki_agent",
    instructions = search_instructions,
    tools=[search,get_page],
    model = 'openai:gpt-4o-mini'
)

In [8]:
from toyaikit.chat import IPythonChatInterface
from toyaikit.chat.runners import PydanticAIRunner

In [9]:
runner = PydanticAIRunner(
    chat_interface=IPythonChatInterface(),
    agent=wiki_agent
)

In [10]:
await runner.run()

You: where do capybaras live


You: stop


Chat ended.
